<a href="https://colab.research.google.com/github/imdeepmind/CrowdCount/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!mkdir data
!unzip -qq /content/drive/My\ Drive/Projects/Crowd\ Count/Dataset/dataset.zip -d ./data

mkdir: cannot create directory ‘data’: File exists


In [0]:
!mkdir data/processed

In [0]:
from os.path import join
import cv2

def process_images(read_folder, save_folder, file):
    read_path = join(read_folder, file)
    save_path = join(save_folder, file)

    img = cv2.imread(read_path, 0)
    img = cv2.resize(img, (555,555))

    cv2.imwrite(save_path, img)

    return save_path

In [6]:
import pandas as pd

data = pd.read_csv("data/labels.csv")
labels = data["count"].values
print(labels[0:10])

[35 41 41 44 41 41 35 36 27 24]


In [9]:
from os import listdir
from tqdm import tqdm

read_folder = "data/frames/"
save_folder = "data/processed/"

images = listdir("data/frames/")

dataset = []

for index, image in tqdm(enumerate(images)):
    save_path = process_images(read_folder, save_folder, image)
    dataset.append([save_path, labels[index]])

2000it [00:11, 168.63it/s]


In [10]:
print(dataset[0:10])

[['data/processed/seq_001236.jpg', 35], ['data/processed/seq_000073.jpg', 41], ['data/processed/seq_000554.jpg', 41], ['data/processed/seq_001469.jpg', 44], ['data/processed/seq_001444.jpg', 41], ['data/processed/seq_000375.jpg', 41], ['data/processed/seq_001291.jpg', 35], ['data/processed/seq_000272.jpg', 36], ['data/processed/seq_000708.jpg', 27], ['data/processed/seq_000313.jpg', 24]]


In [11]:
import pandas as pd

df = pd.DataFrame(dataset, columns=["path", "count"])

print(df.head())

                            path  count
0  data/processed/seq_001236.jpg     35
1  data/processed/seq_000073.jpg     41
2  data/processed/seq_000554.jpg     41
3  data/processed/seq_001469.jpg     44
4  data/processed/seq_001444.jpg     41


In [0]:
df.to_csv("data/processed.csv", index=False)

In [15]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.15, random_state=1969)

print(train_data.shape, test_data.shape)

(1700, 2) (300, 2)


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rotation_range=360,
    rescale=1./255,
    horizontal_flip=True,
	vertical_flip=True,
    validation_split=0.2
)

test_generator = ImageDataGenerator(rescale=1./255)

In [17]:
train_data_gen = train_generator.flow_from_dataframe(
    train_data, 
    x_col="path", 
    y_col="count",
    target_size=(555,555),
    color_mode="grayscale",
    class_mode="other"
)

test_data_gen = test_generator.flow_from_dataframe(
    test_data, 
    x_col="path", 
    y_col="count",
    target_size=(555,555),
    color_mode="grayscale",
    class_mode="other"
)

Found 1700 validated image filenames.
Found 300 validated image filenames.
